## This Notebook is for geometrics related searches and shows how the requests can be built

### Initialize

In [ ]:
import os
import sys
from datetime import datetime

import pandas as pd
import requests

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
import src.esh_objects as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path

base_url = get_base_url()
root_path = get_root_path()
selected = sys.path[0].split(os.sep)[-2]



def execute_search_query(query):
    r = requests.post(f'{base_url}/v1/query/{selected}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[ ['entityDesc', 'position','street', 'houseNumber', 'city', 'postalCode', 'country']]
            df.rename(columns={'entityDesc': 'Description',
                  'position': 'Position', 'street': 'Street', 'houseNumber': 'House Number', 'city': 'City',
                           'postalCode': 'Postal Code', 'country': 'Country'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())


## Search for position Latitude='51.38026158483485'

In [ ]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Location'),
            esh.Comparison(
                property=esh.Property(property='position'),
                operator=esh.ComparisonOperator.Search,
                value=esh.Point(
                    coordinates=[51.40168501692694, 0.02059375141978827]))]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))

## Search for Person and its address